In [23]:
# Download and import libraries

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [4]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TextStreamer
import torch

In [5]:
max_seq_length = 2048  # Maximum sequence length for the model input
dtype = None           # Use auto detection for data type (e.g., Float16 for some GPUs)
load_in_4bit = True    # Enable 4-bit quantization to reduce VRAM usage

# Load Model

In [6]:
model_name = "johnjehiel/PH-LLM-Llama-3.1-8B-Instruct-bnb-4bit"


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token=HF_TOKEN  # Use your Hugging Face token for authentication
)


FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [7]:
prompt_template = """Below is an instruction that describes a task, paired with an optional input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [18]:
def produce_response(input_prompt):
    inputs = tokenizer([
            prompt_template.format(
                input_prompt,
                "",
                ""
            )
        ], return_tensors="pt").to("cuda")
    
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    output = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        streamer=text_streamer,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id,
    )

# Testing / Inference 

# Type A1

In [19]:
produce_response('''You are a sleep medicine expert. You are given the following sleep data.
    The user is Male, 45 years old.
    Sleep Summary:
    Bedtime: 2021-04-10 23:30:00
    Wakeup time: 2021-04-11 06:30:00
    Sleep duration: 7.0
    Sleep efficiency: 0.93
    REM sleep percentage: 22
    Deep sleep percentage: 55
    Light sleep percentage: 23
    Awakenings: 1.0
    Caffeine consumption: 2.0
    Alcohol consumption: 1.0
    Smoking status: No
    Exercise frequency: 4.0
    
    List the most important insights. Identify all of the patterns of data that are likely out of the preferred range. Make sure to consider various sleep health dimensions: Routine, Sleep Quality, Alertness, Timing, Efficiency, and Duration. Add a heading for each dimension. Optionally add a heading called Other for any additional factors. For Routine, evaluate the consistency of bedtime, wake time, and midsleep point. For Sleep Quality, analyze the durations of light, deep, and REM sleep as well as any wake time after sleep onset. For Alertness, assess nap frequency and duration. For Timing, examine bedtime and wake time patterns (including weekend vs. workday differences). For Efficiency, compare sleep efficiency and related measures to similar users. For Duration, review overall sleep duration with corresponding percentiles. Be concise and avoid generic statements.
    '''
)

# Sleep Insights Report

Here's an analysis of your sleep data, focusing on key areas for improvement:

**Routine**

*   **Bedtime (11:30 PM) and wake time (6:30 AM) are consistent, indicating a stable sleep schedule.**

**Sleep Quality**

*   **Deep sleep (55%) is significantly higher than the typical range (13-23%), which may indicate sleep fragmentation or other underlying issues.** 
*   **Light sleep (23%) is lower than the expected range (50-60%), which could be related to the high deep sleep percentage.** 
*   **REM sleep (22%) is within the normal range (20-25%).** 
*   **One awakening is within the typical range (1-2 awakenings per night).**

**Alertness**

*   **No naps were reported, which is ideal for maintaining a stable sleep-wake cycle.**

**Timing**

*   **A 11:30 PM bedtime and 6:30 AM wake time are within a reasonable range.** 

**Efficiency**

*   **Sleep efficiency (93%) is excellent, placing you above the 95th percentile.** 

**Duration**

*   **Sleep duration (7 ho

# Type A6

In [20]:
produce_response("""You are a NSCA and ACSM board-certified fitness trainer who specializes in athlete training performance and recovery.
    Age: 42  
    Gender: Male  
    Sleep Duration: 6.5  
    Quality of Sleep: 7  
    Physical Activity Level: 45  
    Stress Level: 6  
    BMI Category: Overweight  
    Blood Pressure: 125/80  
    Heart Rate: 75  
    Daily Steps: 5000
    
    Are there any special precautions that should be taken into account when recommending a fitness program to avoid injury? Comment if the trainee has exceptional demographics that require special considerations. Write a single sentence. Avoid mentioning diseases.
    """
)

Given the trainee's overweight BMI and slightly elevated blood pressure, a gradual progression of intensity and volume is crucial to prevent overexertion.
<|eot_id|>


# Type A2

In [21]:
produce_response("""You are a sleep medicine expert. You are given the following sleep data.
    The user is Female, 60 years old.
    Sleep Summary:
    Bedtime: 2021-09-15 22:00:00
    Wakeup time: 2021-09-16 06:30:00
    Sleep duration: 8.5
    Sleep efficiency: 0.92
    REM sleep percentage: 22
    Deep sleep percentage: 65
    Light sleep percentage: 13
    Awakenings: 1.5
    Caffeine consumption: 0.0
    Alcohol consumption: 0.0
    Smoking status: No
    Exercise frequency: 3.0
    
    Based on the data, we can get the following insights:
    ## Routine  
    - **Your bedtime (10:00 PM) and wake-up time (6:30 AM) are consistent and support a regular sleep routine.**  
    - **Maintaining this schedule promotes a well-aligned circadian rhythm.**
    
    ## Sleep Quality  
    - **REM sleep (22%) is optimal, while deep sleep (65%) is higher than usual, potentially indicating an imbalance in sleep stage distribution.**  
    - **Light sleep (13%) is notably low, which might suggest atypical cycling during the night.**  
    - **1.5 awakenings indicate minor sleep disturbances.**
    
    ## Alertness  
    - **Absence of daytime naps is positive, yet slight disturbances may affect overall alertness.**
    
    ## Timing  
    - **A midsleep point around 2:15 AM supports natural circadian rhythms, aiding daytime performance.**
    
    ## Efficiency  
    - **A sleep efficiency of 92% is commendable, though slight improvements may be beneficial.**
    
    ## Duration  
    - **An 8.5-hour duration is excellent, meeting the recommended range for adults.**
    
    ## Key Takeaways  
    - **While your sleep routine and duration are strong, the imbalance in sleep stages—especially low light sleep—warrants tracking over the next month.**  
    - **Consider subtle adjustments in pre-sleep routines to potentially normalize the sleep stage distribution.**
    
    Causes:  
    1. **Extrinsic Factors (Likely):** Environmental factors may contribute to the unusual distribution of sleep stages.  
    2. **Homeostatic Drive (Possible):** The high deep sleep percentage might hint at compensatory adjustments in sleep architecture.  
    3. **Circadian Rhythm (Unlikely):** Your consistent sleep timing makes circadian disruption less likely.  
    4. **Psychophysiologic Hyperarousal (Unlikely):** Minimal awakenings support a calm sleep state.
    
    What recommendation(s) can you provide to help balance your sleep stages, and what is one detail about your pre-sleep routine that could be fine-tuned in the next month?
    """
)

Okay, let's break down what you can do to optimize your sleep.

**Recommendations:**

1.  **Track Your Sleep Stages:** For the next month, use a sleep tracker (like a wearable or an app) to monitor your sleep stages, including REM, deep, light, and wake time. This will help you understand how your sleep architecture changes over time.
2.  **Adjust Your Pre-Sleep Routine:** Pay attention to your pre-sleep routine. If you're consistently going to bed at 10:00 PM, consider making small adjustments to your evening activities. For example, try reading a book instead of watching a screen for 30 minutes before bed. This might help you transition into a more balanced sleep state.
3.  **Maintain Your Consistent Sleep Schedule:** Continue to maintain your consistent sleep schedule, even on weekends. This will help reinforce your body's natural sleep-wake cycle.

**One Key Detail to Fine-Tune in Your Pre-Sleep Routine:**

*   **Screen Time Before Bed:** Try to avoid screens (phones, tablets, comp

# Type A3

In [24]:
produce_response("""
    You are a sleep medicine expert. You are given the following sleep data.
    The user is Female, 40 years old.
    Sleep Summary:
    Bedtime: 2021-09-01 22:15:00
    Wakeup time: 2021-09-02 06:45:00
    Sleep duration: 8.5
    Sleep efficiency: 0.92
    REM sleep percentage: 20
    Deep sleep percentage: 35
    Light sleep percentage: 45
    Awakenings: 1.0
    Caffeine consumption: 25.0
    Alcohol consumption: 5.0
    Smoking status: No
    Exercise frequency: 3.0
    
    Based on the data, we can get the following insights:
    
    # Sleep Insights Report
    
    **Routine:**  
    - Your consistent bedtime at 10:15 PM and wake-up time at 6:45 AM indicate a stable sleep routine.
    
    **Sleep Quality:**  
    - With a sleep efficiency of 92% and only one awakening, your sleep quality is generally good.  
    - REM sleep at 20%, deep sleep at 35%, and light sleep at 45% are within acceptable ranges.
    
    **Alertness:**  
    - Good sleep continuity likely supports strong daytime alertness.
    
    **Timing:**  
    - Your midsleep point (around 3:30 AM) is balanced, fitting a healthy circadian rhythm.
    
    **Efficiency:**  
    - High efficiency suggests that you spend minimal time awake during the night.
    
    **Duration:**  
    - An 8.5-hour sleep duration exceeds the minimum recommendation for adults.
    
    **Other:**  
    - Moderate caffeine and alcohol consumption are within acceptable levels for your age.
    
    Causes:  
    1. Extrinsic Factors (Unlikely): Your moderate caffeine and alcohol intake appear acceptable.  
    2. Psychophysiologic Hyperarousal (Unlikely): Minimal awakenings and high efficiency indicate a calm state.  
    3. Circadian Rhythm (Very Likely): Your sleep timing is well aligned, confirming a strong circadian rhythm.  
    4. Homeostatic Drive (Possible): While sleep duration is good, further data would help ensure consistency.
    
    What recommendation(s) can you provide to maintain your healthy sleep habits, and is there any aspect of your pre-sleep routine you’d like to optimize in the next month?
""")

Okay, let's break down your sleep data and provide recommendations to optimize your sleep habits.

**Key Takeaways:**

*   Your sleep efficiency is very high at 92%, indicating you spend most of your time in bed asleep.
*   You have a stable sleep routine with a consistent bedtime and wake-up time.
*   Your sleep duration is 8.5 hours, which is above the recommended minimum for adults.
*   Your REM sleep percentage (20%) is within the normal range, as is your deep sleep percentage (35%).
*   You have a moderate amount of light sleep (45%), which is typical.
*   You experience only one awakening, suggesting good sleep continuity.
*   Your caffeine consumption (25mg) and alcohol intake (5 units) are moderate and unlikely to be a significant issue.

**Recommendations:**

1.  **Maintain Your Sleep Routine:** Continue to prioritize a consistent bedtime (10:15 PM) and wake-up time (6:45 AM). This will help regulate your circadian rhythm and promote better sleep.
2.  **Optimize Your Sleep Env